Import packages and functions

In [ ]:
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as f
import matplotlib.pyplot as plt

# Load data from table into dataframe
session = get_active_session()
taxi_raw_df = session.table("NY_TAXI_DATA.CLEAN.CLEAN_TRAIN")

Pie Chart
Market share by Vendor ID.
There is an opportunity to augment the data with vendor information

In [ ]:

# Aggregate = passenger count by vendor
df_group = taxi_raw_df.group_by("VENDOR_ID").agg(f.sum("PASSENGER_COUNT").alias("VENDOR_ID"))
df_group.show()


# Collect data into a Pandas DataFrame
import pandas as pd
data = df_group.collect()
df_pandas = pd.DataFrame(data, columns=['VENDOR_ID', 'PASSENGER_COUNT'])

# Create the pie chart using Matplotlib
import matplotlib.pyplot as plt

plt.pie(df_pandas['PASSENGER_COUNT'], labels=df_pandas['VENDOR_ID'], autopct='%1.1f%%')
plt.title('Passenger count by Vendor')
plt.show()

Histogram, Bar Chart
Trip duration distribution

In [ ]:
import numpy as np
import seaborn as sns

# settings for seaborn plotting style
sns.set(color_codes=True)
# settings for seaborn plot sizes
sns.set(rc={'figure.figsize':(5,5)})

# aggregate, slicing by trip_duration
df_group2 = taxi_raw_df.group_by("trip_duration").agg(f.count("trip_duration").alias("tcount")).order_by("trip_duration").filter("trip_duration < 80000").filter("tcount > 5")
df_pandas2 = df_group2.to_pandas()

In [ ]:
# prepare bar chart
fig, ax = plt.subplots()
sns.barplot(x='TRIP_DURATION',y='TCOUNT', data=df_pandas2, edgecolor='skyblue')
plt.title('Trip duration distribution')
plt.xlabel('Trip duration, seconds')
plt.ylabel('Number of trips for the duration')
ax.set_xticks([299, 1499, 2999])
plt.show()

In [ ]:
# The same insight - histogram
df_pandas2['TCOUNT'].hist(bins=20)
plt.title("Trip duration histogram")
plt.xlabel('TRIP_DURATION')
plt.ylabel('TCOUNT')
plt.show()

In [ ]:
train = session.table("NY_TAXI_DATA.CLEAN.CLEAN_TRAIN").to_pandas()

Scatter plot, Geographical distribution of taxi service orders

In [ ]:
#scatter plot
sns.scatterplot(data=train, x='PICKUP_LONGITUDE', y='PICKUP_LATITUDE'
                #, hue='VENDOR_ID', size='TRIP_DURATION' # these 2 options don't reveal significant insights, but consume processig power
               ).set(title='Scatter plot', xlabel='Longitude', ylabel='Latitude')

In [ ]:
session = get_active_session()
monthly_trends = session.table("ny_taxi_data.clean.monthly_trend").to_pandas()


In [ ]:
sns.lineplot(data=monthly_trends, x='MONTH_NAME', y='PASSENGER_COUNT_TOTAL')
plt.title("Total passenger count trending over time")
plt.xlabel('2016')

